In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
sys.path.append('/content/gdrive/My Drive/sleep_project')
import os
os.chdir('/content/gdrive/My Drive/sleep_project')

Mounted at /content/gdrive/


In [ ]:
import torch
import numpy as np
# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm
import torch.nn.functional as F
import torch.nn as nn
import random

import math

import pandas as pd

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
target_id = 56
num_it = 1
# output_id = target_id*10 + num_it
output_str = str(target_id) + '_' + str(num_it)

dataset_file = 'Validation_data/iScience_' +  str(target_id) + '_timeseries.csv'

output_file = 'Validation_result/iScience_subject_' + output_str + '_output.csv'
result_file = 'Validation_result/iScience_subject_' + output_str + '_result.csv'
data_file = 'Validation_result/iScience_subject_' + output_str + '_data.csv'


use_gpu = True  # set it to True to use GPU and False to use CPU

if use_gpu:
    torch.cuda.set_device(0)

In [ ]:
Data = pd.read_csv(dataset_file)
Data1 = pd.read_csv(dataset_file)
Data1['Label'] = Data1['SleepStatus'].apply(lambda x: 0 if x == 'Sleep' else 1.)
# Data = Data.drop(columns=['Unnamed: 0'])
Data['denote'] = Data['actigraphy'].apply(lambda x: 0. if np.isnan(x) else 1.)
Data['timespan'] = Data1['timespan']
Data['Label'] = Data1["Label"]
Data

,timespan,SleepStatus,actigraphy,denote,Label
0,600,Wake,0.0,1.0,1.0
1,602,Wake,0.0,1.0,1.0
2,604,Wake,0.0,1.0,1.0
3,606,Wake,0.0,1.0,1.0
4,608,Wake,0.0,1.0,1.0
...,...,...,...,...,...
11770,24140,Wake,402.0,1.0,1.0
11771,24142,Wake,609.0,1.0,1.0
11772,24144,Wake,629.0,1.0,1.0
11773,24146,Wake,1046.0,1.0,1.0


In [ ]:
import time

random.seed(time.clock())

# from random import randint

lol=Data.shape[0]
before = Data[Data['denote'] == 0].shape[0]

for i in range(50):
  getrandom = random.randint(0, lol)
  # getrandom = randint(0, lol)
  # print(getrandom)
  if Data['denote'][getrandom] == 0:
    continue
  else:
    xd = np.random.gamma(3.5, 1.25, 1)[0]
    # xd = random.uniform(0, 6*6)
    Data.loc[getrandom: min(getrandom+xd*10, Data.shape[0]), 'denote'] = 0

after = Data[Data['denote'] == 0].shape[0]
after-before


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


1646

In [ ]:
valid_acti = Data[~Data['actigraphy'].isnull()]
valid_acti = valid_acti['actigraphy']
valid_acti = np.array(valid_acti)

In [ ]:
Data.loc[Data['denote'] == 0, 'actigraphy'] = np.nan
# Data['actigraphy'] = np.array(Data['actigraphy'])

# thresh = np.quantile(Data[~Data['actigraphy'].isnull()], 0.9)
thresh = np.quantile(valid_acti, 0.9)


Data.loc[:, 'actigraphy'] = Data['actigraphy'].apply(lambda x: 1 if x > thresh else x/thresh)
Data

,timespan,SleepStatus,actigraphy,denote,Label
0,600,Wake,0.000000,1.0,1.0
1,602,Wake,0.000000,1.0,1.0
2,604,Wake,0.000000,1.0,1.0
3,606,Wake,0.000000,1.0,1.0
4,608,Wake,0.000000,1.0,1.0
...,...,...,...,...,...
11770,24140,Wake,0.328700,1.0,1.0
11771,24142,Wake,0.497956,1.0,1.0
11772,24144,Wake,0.514309,1.0,1.0
11773,24146,Wake,0.855274,1.0,1.0


In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 32.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633981 sha256=cd1a6112022bbfdf9df62fbead4f3db6ece47891c6a49430f6d9c3dbdd67281d
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
def find_missing_denote(Data):
  denote_features = []
  for ii in Data.columns:
    if 'denote' in ii:
      denote_features.append(ii)

  denote = Data[denote_features]
  denote = np.array(denote, dtype = float)
  denote = np.transpose(denote)

  denote = Data['denote']
  height = len(Data)
  num_days = np.ceil(height/(24*60))
  num_days = int(num_days)
  denote_pad = []

  for i in range(1,num_days+1):
    if i != num_days:
      idx_prev = (i-1)*60*24
      idx_curr = i*60*24
      indices = np.array(range(idx_prev, idx_curr))
      day_curr = np.take(denote, indices)
      if i == 1:
        denote_pad = day_curr
      elif i != 1:
        denote_pad = np.vstack((denote_pad, day_curr))

    elif i == num_days:
      idx_prev = (i-1)*60*24
      idx_curr = int(denote.size)
      indices = np.array(range(idx_prev, idx_curr))
      day_curr = np.take(denote, indices)
      miss = np.zeros(60*24*num_days - idx_curr)
      day_curr = np.hstack((day_curr, miss))
      denote_pad = np.vstack((denote_pad, day_curr))

  return denote_pad

In [ ]:
def find_missing_denote2(Data):
  denote_features = []
  for ii in Data.columns:
    if 'denote' in ii:
      denote_features.append(ii)

  denote = Data[denote_features]
  denote = np.array(denote, dtype = float)
  denote = np.transpose(denote)

  denote = Data['denote']
  height = len(Data)*2
  num_days = np.ceil(height/(24*60))
  num_days = int(num_days)
  denote_pad = []

  for i in range(1,num_days+1):
    if i != num_days:
      idx_prev = int((i-1)*60*24/2)
      idx_curr = int(i*60*24/2)
      indices = np.array(range(idx_prev, idx_curr))
      day_curr = np.take(denote, indices)
      if i == 1:
        denote_pad = day_curr
      elif i != 1:
        denote_pad = np.vstack((denote_pad, day_curr))

    elif i == num_days:
      idx_prev = int((i-1)*60*24/2)
      idx_curr = int(denote.size)
      indices = np.array(range(idx_prev, idx_curr))
      day_curr = np.take(denote, indices)
      miss = np.zeros(int(60*24*num_days/2) - idx_curr)
      day_curr = np.hstack((day_curr, miss))
      denote_pad = np.vstack((denote_pad, day_curr))

  return denote_pad

In [ ]:
from surprise import NMF, Dataset, Reader

################################################################################

# function pad_NMF -- apply padding to the sleep label data so that the NMF considers sleep data per day, rather than per hour
def pad_NMF(Data):
    slp_features = []
    for ii in Data.columns:
      if 'Label' in ii:
        slp_features.append(ii)

    slp = Data[slp_features]
    slp = np.array(slp, dtype = float)
    slp = np.transpose(slp)

    slp = Data['Label']
    height = len(Data)
    num_days = np.ceil(height/(24*60))
    num_days = int(num_days)
    slp_pad = []

    for i in range(1,num_days+1):
      if i != num_days:
        idx_prev = (i-1)*60*24
        idx_curr = i*60*24
        indices = np.array(range(idx_prev, idx_curr))
        day_curr = np.take(slp, indices)
        if i ==1:
          slp_pad = day_curr
        elif i != 1:
          slp_pad = np.vstack((slp_pad, day_curr))

      elif i == num_days:
        idx_prev = (i-1)*60*24
        idx_curr = np.int(slp.size)
        indices = np.array(range(idx_prev, idx_curr))
        day_curr = np.take(slp, indices)
        miss = np.zeros(60*24*num_days - idx_curr)
        day_curr = np.hstack((day_curr, miss))
        slp_pad = np.vstack((slp_pad, day_curr))

    return slp_pad


In [ ]:
from surprise import NMF, Dataset, Reader

################################################################################

# function pad_NMF -- apply padding to the sleep label data so that the NMF considers sleep data per day, rather than per hour
def pad_NMF2(Data):
    slp_features = []
    for ii in Data.columns:
      if 'Label' in ii:
        slp_features.append(ii)

    slp = Data[slp_features]
    slp = np.array(slp, dtype = float)
    slp = np.transpose(slp)

    slp = Data['Label']
    height = len(Data)*2
    num_days = np.ceil(height/(24*60))
    num_days = int(num_days)
    slp_pad = []

    for i in range(1,num_days+1):
      if i != num_days:
        idx_prev = int((i-1)*60*24/2)
        idx_curr = int(i*60*24/2)
        indices = np.array(range(idx_prev, idx_curr))
        day_curr = np.take(slp, indices)
        if i ==1:
          slp_pad = day_curr
        elif i != 1:
          slp_pad = np.vstack((slp_pad, day_curr))

      elif i == num_days:
        idx_prev = int((i-1)*60*24/2)
        idx_curr = np.int(slp.size)
        indices = np.array(range(idx_prev, idx_curr))
        day_curr = np.take(slp, indices)
        miss = np.zeros(int(60*24*num_days*0.5) - idx_curr)
        day_curr = np.hstack((day_curr, miss))
        slp_pad = np.vstack((slp_pad, day_curr))

    return slp_pad

In [ ]:
from surprise import NMF, Dataset, Reader

def sleep_NMF2(pad_data, Data, num_feat):
  NMF_data = Data['Label']
  slp = pad_data

  # Extract denote features
  denote = find_missing_denote2(Data)
  # Replace labels to avoid float division error
  R = pad_data
  l = len(R)
  h = int(R.size/l)
  for i in range(0,l):
    for j in range(0,h):
      if slp[i,j] == 1:
        R[i,j] = 2
      elif slp[i,j] == 0:
        R[i,j] = 1

      if denote[i,j] == 0:
        R[i,j] = np.nan

  # Set up for NMF
  df = pd.DataFrame(data=R, index=range(R.shape[0]), columns=range(R.shape[1]))
  df = pd.melt(df.reset_index(), id_vars='index', var_name='items', value_name='sleep').dropna(axis=0)
  reader = Reader(rating_scale=(1, 2))
  data = Dataset.load_from_df(df[['index', 'items', 'sleep']], reader)

  k = num_feat # number of feature components
  algo = NMF(n_factors=k)
  trainset = data.build_full_trainset()
  algo.fit(trainset)

  predictions = algo.test(trainset.build_testset()) # predict the known ratings
  R_hat = np.zeros_like(R)
  for uid, iid, true_r, est, _ in predictions:
      R_hat[uid, iid] = est

  predictions = algo.test(trainset.build_anti_testset()) # predict the unknown ratings
  for uid, iid, true_r, est, _ in predictions:
      R_hat[uid, iid] = est

  R_hat = R_hat - 1

  # np.savetxt('R.csv', R, delimiter = ",")

  return R_hat

In [ ]:
from surprise import NMF, Dataset, Reader

# sleep_NMF: Function to implement NMF to sleep labels
# Input: Data -- the imported data before separating them into Label, denote, and actigraphy
#        num_feat -- number of underlying features, i.e., the rank of the lower rank matrix represenataion
def sleep_NMF(pad_data, Data, num_feat):
  NMF_data = Data['Label']
  slp = pad_data

  # Extract denote features
  denote = find_missing_denote(Data)

  # print(denote.shape)

  # Replace labels to avoid float division error
  R = pad_data
  l = len(R)
  h = int(R.size/l)
  for i in range(0,l):
    for j in range(0,h):
      if slp[i,j] == 1:
        R[i,j] = 2
      elif slp[i,j] == 0:
        R[i,j] = 1

      if denote[i,j] == 0:
        R[i,j] = np.nan

  # Set up for NMF
  df = pd.DataFrame(data=R, index=range(R.shape[0]), columns=range(R.shape[1]))
  df = pd.melt(df.reset_index(), id_vars='index', var_name='items', value_name='sleep').dropna(axis=0)
  reader = Reader(rating_scale=(1, 2))
  data = Dataset.load_from_df(df[['index', 'items', 'sleep']], reader)

  k = num_feat # number of feature components
  algo = NMF(n_factors=k)
  trainset = data.build_full_trainset()
  algo.fit(trainset)

  predictions = algo.test(trainset.build_testset()) # predict the known ratings
  R_hat = np.zeros_like(R)
  for uid, iid, true_r, est, _ in predictions:
      R_hat[uid, iid] = est

  predictions = algo.test(trainset.build_anti_testset()) # predict the unknown ratings
  for uid, iid, true_r, est, _ in predictions:
      R_hat[uid, iid] = est

  R_hat = R_hat - 1

  # np.savetxt('R.csv', R, delimiter = ",")
  # print(R.shape)

  return R_hat
################################################################################
# A function to find the optimal number of underlying features
def find_opt_num_feat(pad_data, Data):
  # Extract denote features

  denote = find_missing_denote(Data)

  slp = pad_data

  # Replace labels to avoid float division error

  pred = slp
  pred[denote==0] = np.nan
  min_loss = 1000000000
  opt_num_feat = 0

  # Start for loop to find the optimal number of underlying features
  for iii in range(1,25):
    nmf = sleep_NMF(pad_data, Data, iii)
    diff = np.subtract(nmf[denote==1], pred[denote==1])
    diff = np.square(diff)
    loss = np.sum(diff)

    # print(loss)

    if loss <= min_loss:
      min_loss = loss
      opt_num_feat = iii

  return opt_num_feat


################################################################################


slp_data = pad_NMF(Data)
# num_feat = find_opt_num_feat(slp_data, Data)
# print(slp_data.shape)
num_feat = 20

NMF_data = sleep_NMF(slp_data, Data, num_feat)
NMF_info = NMF_data
NMF_info.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


(9, 1440)

In [ ]:
Data['NMF'] = NMF_info.reshape(-1)[:Data.shape[0]]
Data

,timespan,SleepStatus,actigraphy,denote,Label,NMF
0,600,Wake,0.000000,1.0,1.0,0.945128
1,602,Wake,0.000000,1.0,1.0,0.960049
2,604,Wake,0.000000,1.0,1.0,0.946491
3,606,Wake,0.000000,1.0,1.0,0.988792
4,608,Wake,0.000000,1.0,1.0,0.957145
...,...,...,...,...,...,...
11770,24140,Wake,0.328700,1.0,1.0,0.932757
11771,24142,Wake,0.497956,1.0,1.0,0.931175
11772,24144,Wake,0.514309,1.0,1.0,0.917478
11773,24146,Wake,0.855274,1.0,1.0,0.939328


In [ ]:
# Data['known_label'] = Data['Label']
# Data.loc[Data['denote'] == 0, 'known_label'] = -1
# Data.loc[Data['denote'] == 0, 'actigraphy'] = -1

Data['known_label'] = Data['Label']
Data.loc[Data['denote'] == 0, 'known_label'] = 0
Data.loc[Data['denote'] == 0, 'actigraphy'] = -1

In [ ]:
# for target_id in [54,56,57,58,59,60,61,62,64]:
#   for num_it in range(1,31):
#     output_str = str(target_id) + '_' + str(num_it)
#     dataset_file = 'Validation_data/iScience_' +  str(target_id) + '_timeseries.csv'
#     data_file = 'Validation_result/30_sim/iScience_subject_' + output_str + '_data.csv'
#     # data_file = 'iScience_subject_' + output_str + '_data.csv'

#     Data = pd.read_csv(dataset_file)
#     Data1 = pd.read_csv(dataset_file)
#     Data1['Label'] = Data1['SleepStatus'].apply(lambda x: 0 if x == 'Sleep' else 1.)
#     # Data = Data.drop(columns=['Unnamed: 0'])
#     Data['denote'] = Data['actigraphy'].apply(lambda x: 0. if np.isnan(x) else 1.)
#     Data['timespan'] = Data1['timespan']
#     Data['Label'] = Data1["Label"]

#     random.seed(time.clock())

#     lol=Data.shape[0]

#     for j in range(50):
#       getrandom = random.randint(0, lol)
#       if Data['denote'][getrandom] == 0:
#         continue
#       else:
#         xd = np.random.gamma(3.5, 1.25, 1)[0]
#         Data.loc[getrandom: min(getrandom+xd*10, Data.shape[0]), 'denote'] = 0

#     valid_acti = Data[~Data['actigraphy'].isnull()]
#     valid_acti = valid_acti['actigraphy']
#     valid_acti = np.array(valid_acti)

#     Data.loc[Data['denote'] == 0, 'actigraphy'] = np.nan
#     thresh = np.quantile(valid_acti, 0.9)
#     Data.loc[:, 'actigraphy'] = Data['actigraphy'].apply(lambda x: 1 if x > thresh else x/thresh)
#     slp_data = pad_NMF2(Data)
#     num_feat = 20

#     NMF_data = sleep_NMF2(slp_data, Data, num_feat)
#     NMF_info = NMF_data

#     Data['NMF'] = NMF_info.reshape(-1)[:Data.shape[0]]
#     Data['known_label'] = Data['Label']
#     Data.loc[Data['denote'] == 0, 'known_label'] = 0
#     Data.loc[Data['denote'] == 0, 'actigraphy'] = -1

#     fillmisscol = np.full((Data.shape[0]), 0)

#     i = 0
#     while i < Data.shape[0]:
#       if Data['denote'][i] == 0:
#         counter = 1
#         while i + counter < Data.shape[0] and Data['denote'][i + counter] == 0:
#           counter += 1
#         fillmisscol[i:i+counter] = counter
#         i = i + counter
#       else:
#         i += 1

#     Data['type'] = fillmisscol // 30 + 1
#     Data.loc[ fillmisscol==0 , 'type'] = 0

#     Data = Data.drop(columns=['SleepStatus'])

#     # np.savetxt(data_file, Data)
#     np.savetxt(data_file, Data, delimiter=',')
#     # np.savetxt(data_file, Data, fmt='%s')

In [ ]:
Data

,timespan,SleepStatus,actigraphy,denote,Label,NMF,known_label
0,600,Wake,0.000000,1.0,1.0,0.945128,1.0
1,602,Wake,0.000000,1.0,1.0,0.960049,1.0
2,604,Wake,0.000000,1.0,1.0,0.946491,1.0
3,606,Wake,0.000000,1.0,1.0,0.988792,1.0
4,608,Wake,0.000000,1.0,1.0,0.957145,1.0
...,...,...,...,...,...,...,...
11770,24140,Wake,0.328700,1.0,1.0,0.932757,1.0
11771,24142,Wake,0.497956,1.0,1.0,0.931175,1.0
11772,24144,Wake,0.514309,1.0,1.0,0.917478,1.0
11773,24146,Wake,0.855274,1.0,1.0,0.939328,1.0


In [ ]:
data_file

'Validation_result/iScience_subject_56_1_data.csv'

In [ ]:
from surprise import NMF, Dataset, Reader
def generate_features(Data1):

  Data1.loc[Data1['denote'] == 0, 'actigraphy'] = np.nan

  thresh = np.quantile(Data1[~Data1['actigraphy'].isnull()], 0.9)
  Data1.loc[:, 'actigraphy'] = Data1['actigraphy'].apply(lambda x: 1 if x > thresh else x/thresh)

  slp_data = pad_NMF(Data1)
  num_feat = 20

  NMF_data = sleep_NMF(slp_data, Data1, num_feat)

  Data1['NMF'] = NMF_data.reshape(-1)[:Data1.shape[0]]

  Data1['known_label'] = Data1['Label']
  Data1.loc[Data1['denote'] == 0, 'known_label'] = 0
  Data1.loc[Data1['denote'] == 0, 'actigraphy'] = -1

  return Data1



In [ ]:
trainusers = []
for i in range(18, 46):
  if i == 20:
    continue

  datasetfile = 'SWSI0' + str(i) + '_timeseries_masked.csv'
  datalol = pd.read_csv(datasetfile)

  dataset_file_1 = 'new_data/SWSI0' + str(i) + '_timeseries.csv'
  datalol1 = pd.read_csv(dataset_file_1)
  datalol1['Label'] = datalol1['SleepStatus'].apply(lambda x: 0 if x == 'Sleep' else 1.)

  datalol['Label'] = datalol1["Label"]
  datalol = datalol.drop(columns=['Unnamed: 0'])

  trainusers.append(generate_features(datalol))

trainusers = pd.concat(trainusers)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
fillmisscol = np.full((Data.shape[0]), 0)

i = 0
while i < Data.shape[0]:
  if Data['denote'][i] == 0:
    counter = 1
    while i + counter < Data.shape[0] and Data['denote'][i + counter] == 0:
      counter += 1
    fillmisscol[i:i+counter] = counter
    i = i + counter
  else:
    i += 1

Data['type'] = fillmisscol // 30 + 1
Data.loc[ fillmisscol==0 , 'type'] = 0
Data

,timespan,SleepStatus,actigraphy,denote,Label,NMF,known_label,type
0,600,Wake,0.000000,1.0,1.0,0.945128,1.0,0
1,602,Wake,0.000000,1.0,1.0,0.960049,1.0,0
2,604,Wake,0.000000,1.0,1.0,0.946491,1.0,0
3,606,Wake,0.000000,1.0,1.0,0.988792,1.0,0
4,608,Wake,0.000000,1.0,1.0,0.957145,1.0,0
...,...,...,...,...,...,...,...,...
11770,24140,Wake,0.328700,1.0,1.0,0.932757,1.0,0
11771,24142,Wake,0.497956,1.0,1.0,0.931175,1.0,0
11772,24144,Wake,0.514309,1.0,1.0,0.917478,1.0,0
11773,24146,Wake,0.855274,1.0,1.0,0.939328,1.0,0


In [ ]:
df = Data.copy()
print(df.shape)

(11775, 8)


In [ ]:
featuresdata = df[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
target = df['Label']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
thresh = int(df.shape[0]*0.8)

X_train = featuresdata[:thresh, :]
y_train = target[:thresh]

X_test = featuresdata[thresh:, :]
y_test = target[thresh:]

X_train = torch.tensor(X_train)#.to(device)
X_test = torch.tensor(X_test)#.to(device)
y_train = torch.tensor(y_train.values)#.to(device)
y_test = torch.tensor(y_test.values)#.to(device)

X_train = trainusers[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
y_train = trainusers['Label']

thresh = int(X_train.shape[0]*0.8)

X_valid = X_train[thresh:, :]
y_valid = y_train[thresh:]

X_train = X_train[:thresh, :]
y_train = y_train[:thresh]

X_test = df[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
y_test = df['Label']


X_train = torch.tensor(X_train)#.to(device)
X_test = torch.tensor(X_test)#.to(device)
X_valid = torch.tensor(X_valid)

y_train = torch.tensor(y_train.values)#.to(device)
y_test = torch.tensor(y_test.values)#.to(device)
y_valid = torch.tensor(y_valid.values)#.to(device)

In [ ]:
print(X_train.shape)
print(X_test.shape)

torch.Size([1770984, 4])
torch.Size([11775, 4])


In [ ]:
class simple_model(nn.Module):
  def __init__(self, dim):

    super(simple_model, self).__init__()

    self.dim=dim

    self.linear1 = nn.Linear(dim, dim)

    self.linear2 = nn.Linear(dim, dim)
    self.linear3 = nn.Linear(dim, 2)

  def forward(self, x):
    out = torch.nn.ReLU()(self.linear1(x))

    out = torch.nn.ReLU()(self.linear2(out))
    out = torch.sigmoid(self.linear3(out))

    return out

In [ ]:
sleep_weight = y_train.shape[0]/ (2* y_train[y_train == 0].shape[0])
wake_weight =  y_train.shape[0]/ (2* y_train[y_train == 1].shape[0])

print(sleep_weight)
print(wake_weight)

1.4906394404370114
0.7523622518902688


In [ ]:
print(y_train[y_train == 0].shape[0])
print(y_train.shape[0])

print(y_train[y_train == 1].shape[0])
print(y_train.shape[0])

594035
1770984
1176949
1770984


In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, features, labels):
        'Initialization'
        self.labels = labels
        self.features = features

  def __len__(self):
        'Denotes the total number of samples'
        return self.features.shape[0]

  def __getitem__(self, index):
        'Generates one sample of data'

        # Load data and get label
        X = self.features[index, :]
        y = self.labels[index]

        return X, y

In [ ]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 2}

training_set = Dataset(X_train, y_train)
training_generator = torch.utils.data.DataLoader(training_set, **params)

valid_set = Dataset(X_valid, y_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, **params)

In [ ]:
model = simple_model(4).to(device)
optimizer = torch.optim.Adam(params=model.parameters())

early_stopping = EarlyStopping(patience=10, verbose=True)
valid_losses = []

for i in range(150):
  for j, (X, y) in enumerate(training_generator):
    optimizer.zero_grad()

    probs = model(X.float().to(device)).float()
    loss = torch.nn.CrossEntropyLoss()(probs, y.type(torch.LongTensor).to(device))

    loss = torch.nn.CrossEntropyLoss(reduction='none')(probs, y.type(torch.LongTensor).to(device))
    loss[y==0] *= sleep_weight
    loss[y==1] *= wake_weight
    loss = torch.mean(loss)

    loss.backward()
    optimizer.step()

  model.eval()
  for j, (X, y) in enumerate(valid_generator):

    probs = model(X.float().to(device)).float()
    loss = torch.nn.CrossEntropyLoss()(probs, y.type(torch.LongTensor).to(device))

    loss = torch.nn.CrossEntropyLoss(reduction='none')(probs, y.type(torch.LongTensor).to(device))
    loss[y==0] *= sleep_weight
    loss[y==1] *= wake_weight
    loss = torch.mean(loss)

    valid_losses.append(loss.item())

  valid_loss = np.average(valid_losses)
  valid_losses = []

  early_stopping(valid_loss, model)
  if early_stopping.early_stop:
      print("Early stopping")
      break


Validation loss decreased (inf --> 0.324094).  Saving model ...
Validation loss decreased (0.324094 --> 0.324081).  Saving model ...
Validation loss decreased (0.324081 --> 0.323794).  Saving model ...
Validation loss decreased (0.323794 --> 0.323696).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.323696 --> 0.323683).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping


In [ ]:
result = model(X_test.float().to(device))[:, 1]

dfvalid = pd.DataFrame()
dfvalid['prob'] = result.detach().cpu().numpy()
dfvalid['ground'] = y_test.cpu().numpy()
dfvalid['type'] = df['type']#df.iloc[thresh:df.shape[0]]['type'].values
dfvalid['pred'] = dfvalid['prob'].apply(lambda x: 1 if x>=0.5 else 0)
dfvalid['acti'] = df['actigraphy'].values#df.iloc[thresh:, :]['actigraphy'].values
dfvalid['NMF'] = df['NMF'].values#df.iloc[thresh:, :]['actigraphy'].values



In [ ]:
np.savetxt(output_file, dfvalid, delimiter=",")
# output file format
# column 1: timestamp, column 2: probability, column 3: ground truth,
# column 4: missing type, column 5: predicted label (actigraphy label),
# column 6: activity measurement, column 7: NMF

# np.savetxt(result_file, result_data, delimiter=",")
# column 1: type 1, column 2: type 2, column 3: type 3, column 4: type 4
# row 1: Total sleep, row 2: sleep accuracy, row 3: Total wake, row 4: wake accuracy

In [ ]:
for i in [54,56,57,58,59,60,61,62,64]:
  # df = pd.read_csv('/content/gdrive/My Drive/sleep_project/new_data/SWSI0' + str(i) + '_timeseries.csv')
  df = pd.read_csv('/content/gdrive/My Drive/sleep_project/Validation_data/iScience_' + str(i) + '_timeseries.csv')
  df['Label'] = df['SleepStatus'].apply(lambda x: 0 if x == 'Sleep' else 1.)
  y_test = df['Label']
  y_test = torch.tensor(y_test.values)#.to(device)
  for k in range(1,31):
    id = str(i) + '_' + str(k)

    output_file = 'Validation_result/iScience_subject_' + id + '_output.csv'
    data = np.array(pd.read_csv('/content/gdrive/My Drive/sleep_project/Validation_result/30_sim/iScience_subject_' + id + '_data.csv', header=None))

    Data = pd.DataFrame()
    Data['timespan'] = data[:,0]
    Data['actigraphy'] = data[:,1]
    Data['denote'] = data[:,2]
    Data['Label'] = data[:,3]
    Data['NMF'] = data[:,4]
    Data['known_label'] = data[:,5]
    Data['type'] = data[:,6]

    df = Data.copy()
    featuresdata = df[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
    target = df['Label']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    thresh = int(df.shape[0]*0.8)
    X_train = featuresdata[:thresh, :]
    y_train = target[:thresh]
    X_test = featuresdata[thresh:, :]
    y_test = target[thresh:]
    X_train = torch.tensor(X_train)#.to(device)
    X_test = torch.tensor(X_test)#.to(device)
    y_train = torch.tensor(y_train.values)#.to(device)
    y_test = torch.tensor(y_test.values)#.to(device)
    X_train = trainusers[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
    y_train = trainusers['Label']
    thresh = int(X_train.shape[0]*0.8)
    X_valid = X_train[thresh:, :]
    y_valid = y_train[thresh:]
    X_train = X_train[:thresh, :]
    y_train = y_train[:thresh]
    X_test = df[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
    y_test = df['Label']
    X_train = torch.tensor(X_train)#.to(device)
    X_test = torch.tensor(X_test)#.to(device)
    X_valid = torch.tensor(X_valid)
    y_train = torch.tensor(y_train.values)#.to(device)
    y_test = torch.tensor(y_test.values)#.to(device)
    y_valid = torch.tensor(y_valid.values)#.to(device)

    result = model(X_test.float().to(device))[:, 1]
    dfvalid = pd.DataFrame()
    dfvalid['prob'] = result.detach().cpu().numpy()
    dfvalid['ground'] = y_test.cpu().numpy()
    dfvalid['type'] = df['type']#df.iloc[thresh:df.shape[0]]['type'].values
    dfvalid['pred'] = dfvalid['prob'].apply(lambda x: 1 if x>=0.5 else 0)
    dfvalid['acti'] = df['actigraphy'].values#df.iloc[thresh:, :]['actigraphy'].values
    dfvalid['NMF'] = df['NMF'].values#df.iloc[thresh:, :]['actigraphy'].values

    np.savetxt(output_file, dfvalid, delimiter=",")



In [ ]:
for k in range(1,4):
  for j in range(1,11):
    if k == 1:
      Data = pd.read_csv('/content/gdrive/My Drive/sleep_project/Validation_data/Validation_OSY_timeseries.csv')
      # output_file = 'Validation_result/Validation_OSY_output.csv'
      out_file = 'OSY'
    elif k == 2:
      Data = pd.read_csv('/content/gdrive/My Drive/sleep_project/Validation_data/Validation_LHY_timeseries.csv')
      # output_file = 'Validation_result/Validation_LHY_output.csv'
      out_file = 'LHY'
    elif k == 3:
      Data = pd.read_csv('/content/gdrive/My Drive/sleep_project/Validation_data/Validation_KJW_timeseries.csv')
      # output_file = 'Validation_result/Validation_KJW_output.csv'
      out_file = 'KJW'

    Data1 = Data
    Data1['Label'] = Data1['SleepStatus'].apply(lambda x: 0 if x == 'Sleep' else 1.)
    # Data = Data.drop(columns=['Unnamed: 0'])
    Data['denote'] = Data['actigraphy'].apply(lambda x: 0. if np.isnan(x) else 1.)
    Data['timespan'] = Data1['timespan']
    Data['Label'] = Data1["Label"]
    Data = Data.drop(columns=['SleepStatus'])
    num_it = str(j)
    output_file = 'Validation_result/Validation_' + out_file + '_' + num_it + '_output.csv'

    import time

    random.seed(time.clock())

    lol=Data.shape[0]

    for ii in range(100):
      getrandom = random.randint(0, lol)
      # getrandom = randint(0, lol)
      # print(getrandom)
      if Data['denote'][getrandom] == 0:
        continue
      else:
        xd = np.random.gamma(3.5, 1.25, 1)[0]
        # xd = random.uniform(0, 6*6)
        Data.loc[getrandom: min(getrandom+xd*10, Data.shape[0]), 'denote'] = 0

    valid_acti = Data[~Data['actigraphy'].isnull()]
    valid_acti = valid_acti['actigraphy']
    valid_acti = np.array(valid_acti)

    Data.loc[Data['denote'] == 0, 'actigraphy'] = np.nan
    thresh = np.quantile(valid_acti, 0.9)
    Data.loc[:, 'actigraphy'] = Data['actigraphy'].apply(lambda x: 1 if x > thresh else x/thresh)
    slp_data = pad_NMF(Data)
    num_feat = 20

    from surprise import NMF, Dataset, Reader

    NMF_data = sleep_NMF(slp_data, Data, num_feat)
    NMF_info = NMF_data

    Data['NMF'] = NMF_info.reshape(-1)[:Data.shape[0]]
    Data['known_label'] = Data['Label']
    Data.loc[Data['denote'] == 0, 'known_label'] = 0
    Data.loc[Data['denote'] == 0, 'actigraphy'] = -1

    fillmisscol = np.full((Data.shape[0]), 0)

    iii = 0
    while iii < Data.shape[0]:
      if Data['denote'][iii] == 0:
        counter = 1
        while iii + counter < Data.shape[0] and Data['denote'][iii + counter] == 0:
          counter += 1
        fillmisscol[iii:iii+counter] = counter
        iii = iii + counter
      else:
        iii += 1

    Data['type'] = fillmisscol // 30 + 1
    Data.loc[ fillmisscol==0 , 'type'] = 0


    df = Data.copy()
    featuresdata = df[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
    target = df['Label']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    thresh = int(df.shape[0]*0.8)
    X_train = featuresdata[:thresh, :]
    y_train = target[:thresh]
    X_test = featuresdata[thresh:, :]
    y_test = target[thresh:]
    X_train = torch.tensor(X_train)#.to(device)
    X_test = torch.tensor(X_test)#.to(device)
    y_train = torch.tensor(y_train.values)#.to(device)
    y_test = torch.tensor(y_test.values)#.to(device)
    X_train = trainusers[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
    y_train = trainusers['Label']
    thresh = int(X_train.shape[0]*0.8)
    X_valid = X_train[thresh:, :]
    y_valid = y_train[thresh:]

    X_train = X_train[:thresh, :]
    y_train = y_train[:thresh]
    X_test = df[['actigraphy', 'denote', 'NMF', 'known_label']].to_numpy()
    y_test = df['Label']
    X_train = torch.tensor(X_train)#.to(device)
    X_test = torch.tensor(X_test)#.to(device)
    X_valid = torch.tensor(X_valid)
    y_train = torch.tensor(y_train.values)#.to(device)
    y_test = torch.tensor(y_test.values)#.to(device)
    y_valid = torch.tensor(y_valid.values)#.to(device)

    result = model(X_test.float().to(device))[:, 1]
    dfvalid = pd.DataFrame()
    dfvalid['prob'] = result.detach().cpu().numpy()
    dfvalid['ground'] = y_test.cpu().numpy()
    dfvalid['type'] = df['type']#df.iloc[thresh:df.shape[0]]['type'].values
    dfvalid['pred'] = dfvalid['prob'].apply(lambda x: 1 if x>=0.5 else 0)
    dfvalid['acti'] = df['actigraphy'].values#df.iloc[thresh:, :]['actigraphy'].values
    dfvalid['NMF'] = df['NMF'].values#df.iloc[thresh:, :]['actigraphy'].values

    np.savetxt(output_file, dfvalid, delimiter=",")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
result_data = np.zeros((4,4))

In [ ]:
# for i in range(1, 5):
for i in range(1, 5):

  print('------------------------------------')
  all0 = dfvalid[(dfvalid['ground'] == 0) & (dfvalid['type'] == i) ].shape[0]
  all1 = dfvalid[(dfvalid['ground'] == 1) & (dfvalid['type'] == i) ].shape[0]
  tp = dfvalid[(dfvalid['ground'] == 0) & (dfvalid['type'] == i) & (dfvalid['pred'] == 0)].shape[0]
  tn = dfvalid[(dfvalid['ground'] == 1) & (dfvalid['type'] == i) & (dfvalid['pred'] == 1)].shape[0]

  print('Type ', i)
  print('Total_sleep: ', all0)
  result_data[0,i-1] = all0

  if all0 > 0:
    print('AccSleep: ', tp/all0)
    result_data[1,i-1] = tp/all0
  else:
    result_data[1,i-1] = np.nan

  print('Total_wake: ',  all1)
  result_data[2,i-1] = all1

  if all1 > 0:
    print('AccWake: ', tn/all1)
    result_data[3,i-1] = tn/all1
  else:
    result_data[3,i-1] = np.nan

In [ ]:
print(result_data)

In [ ]:
from sklearn import metrics

roc_input = dfvalid[(dfvalid['type'] != 0) & dfvalid['type'] <= 48]
# roc_input = dfvalid[(dfvalid['type'] == 1)]
# roc_input = dfvalid[(dfvalid['type'] == 4) | (dfvalid['type'] == 3)]
ground = roc_input['ground']
prob = roc_input['prob']
fpr, tpr, thresholds = metrics.roc_curve(ground, prob, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
result_file

In [ ]:
import matplotlib.pyplot as plt

ax = dfvalid.iloc[2500:4000, :].plot(y=['type', 'pred', 'ground', 'NMF'], legend=True, title='Accuracy', figsize = (30, 10), ylabel='Accuracy', fontsize=16)

ax.legend(fontsize=16)
ax.set_ylabel('Accuracy',fontdict={'fontsize':16})
ax.set_xlabel('Timespan-5min',fontdict={'fontsize':16})

In [ ]:
from sklearn import metrics

roc_input = dfvalid[(dfvalid['type'] != 0)]
# roc_input = dfvalid[(dfvalid['type'] == 2)]
# roc_input = dfvalid[(dfvalid['type'] == 5) | (dfvalid['type'] == 6)]
ground = roc_input['ground']
prob = roc_input['prob']
fpr, tpr, thresholds = metrics.roc_curve(ground, prob, pos_label=1)
metrics.auc(fpr, tpr)